In [1]:
import numpy as np

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops

from scipy.stats import wasserstein_distance_nd

import params

np.random.seed(0)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/ipykernel/kernelapp.py", lin

In [2]:
experiment = 'synth1_gnd'

In [3]:
# import subprocess
# subprocess.run(['python', 'params.py'])

In [4]:
# Define the radius of the Wasserstein balls (epsilon, delta) and the size for both models.
epsilon         = params.radius[experiment][0]
ll_num_envs     = params.n_envs[experiment][0]

delta           = params.radius[experiment][1]
hl_num_envs     = params.n_envs[experiment][1]

# Define the number of samples per environment. Currently every environment has the same number of samples
num_llsamples   = params.n_samples[experiment][0]
num_hlsamples   = params.n_samples[experiment][1]

In [5]:
Dll = mut.load_samples(experiment)[None][0]
Gll = mut.load_ll_model(experiment)[0]
Ill = mut.load_ll_model(experiment)[1]


Dhl = mut.load_samples(experiment)[None][1]
Ghl = mut.load_hl_model(experiment)[0]
Ihl = mut.load_hl_model(experiment)[1]

omega = mut.load_omega_map(experiment)

num_llvars      = Dll.shape[1]
num_hlvars      = Dhl.shape[1]

In [10]:
ll_coeffs = mut.get_mle_coefficients_gmm(Dll, Gll, weights=None, n_components=num_llvars)
hl_coeffs = mut.get_mle_coefficients_gmm(Dhl, Ghl, weights=None, n_components=num_hlvars)

In [11]:
U_ll_hat, mu_U_ll_hat, Sigma_U_ll_hat = mut.lan_abduction(Dll, Gll, ll_coeffs)
U_hl_hat, mu_U_hl_hat, Sigma_U_hl_hat = mut.lan_abduction(Dhl, Ghl, hl_coeffs)

A_ll = mut.generate_perturbed_datasets(D = U_ll_hat, bound = epsilon, num_envs = ll_num_envs) #Low-level: A_epsilon
A_hl = mut.generate_perturbed_datasets(D = U_hl_hat, bound = delta, num_envs = hl_num_envs) #High-level A_delta

In [19]:
import numpy as np
from scipy.stats import wasserstein_distance_nd

# Sample matrices
matrix1 = np.array([[-0.55136366,  2.25774711, -0.86015407],
                    [ 0.61503798,  1.77686484, -1.14240173],
                    [ 0.4236548 ,  2.36859825, -1.01963526],
                    [ 0.16244389,  2.43970299, -0.1532324 ],
                    [-0.01409638,  1.35641781, -2.70307514],
                    [ 0.4121921 ,  2.12690513, -3.19979686]])

matrix2 = np.array([[-0.40278489,  2.42774661, -1.27318467],
                    [ 0.52018708,  2.03810244, -1.23559032],
                    [ 0.46968178,  2.5194733 , -1.09609204],
                    [ 0.10345296,  2.00214177, -0.52794575],
                    [ 0.31595855,  1.36514995, -2.49965091],
                    [-0.17957056,  2.65735194, -3.58858962]])

# Compute Wasserstein distance between the two multivariate distributions
distance = wasserstein_distance_nd(matrix1, matrix2)

print(f"Wasserstein distance: {distance}")


Wasserstein distance: 0.4651247987640955


In [12]:
LLmodels = {}
for iota in Ill:
    LLmodels[iota] = lanm.LinearAddSCM(Gll, ll_coeffs, iota)
    
HLmodels, Dhl_samples = {}, {}
for eta in Ihl:
    HLmodels[eta] = lanm.LinearAddSCM(Ghl, hl_coeffs, eta)

In [22]:
abstraction_errors             = {}
abstraction_env_errors         = {}
max_env_avg_interv_error_value = -np.inf
max_env_avg_interv_error_key   = None

for lenv in A_ll:
    for henv in A_hl:
        total_ui_error = 0
        num_distros    = len(Ill)

        T  = mut.sample_stoch_matrix(num_hlvars, num_llvars) # sample the abstraction map/matrix

        for iota in Ill:
            llcm   = LLmodels[iota]
            hlcm   = HLmodels[omega[iota]]
            llmech = llcm.compute_mechanism()
            hlmech = hlcm.compute_mechanism()

            lefthh = T @ (llmech @ lenv.T)
            righthh = hlmech @ henv.T
            #print(rig)
            error = wasserstein_distance_nd(lefthh, righthh)
            #error = mut.mat_jsd_distance(T@(llmech @ lenv.T), hlmech @ henv.T)
            #error = mut.mat_ot_wasserstein_distance(T@(llmech @ lenv.T), hlmech @ henv.T)
            #error  = mut.mat_wasserstein_distance(T@(llmech @ lenv.T), hlmech @ henv.T)
            
            #print(error,'\n')
            total_ui_error += error

        avg_interv_error = total_ui_error/num_distros

        if avg_interv_error > max_env_avg_interv_error_value:
            max_env_avg_interv_error_value = avg_interv_error
            max_env_avg_interv_error_key   = (lenv, henv)

        abstraction_errors[str(T)] = avg_interv_error
        #abstraction_env_errors['ll: '+str(ll_environment.means_)+' hl: '+str(hl_environment.means_)] = avg_interv_error

115.78577543617266 

119.11496379782378 

115.78577543617266 

115.78577543617266 

119.11496379782378 

119.11496379782378 

119.11506138389129 

127.95031670149393 

119.11506138389129 

119.11506138389129 

127.95031670149393 

127.95031670149393 

117.88659157284552 

127.31904052360852 

117.88659157284552 

117.88659157284552 

127.31904052360852 

127.31904052360852 

158.31560168930432 

171.14041950368699 

158.31560168930432 

158.31560168930432 

171.14041950368699 

171.14041950368699 

134.20801254736693 

140.6445898831946 

134.20801254736693 

134.20801254736693 

140.6445898831946 

140.6445898831946 

149.43885239259959 

148.80733450633352 

149.43885239259959 

149.43885239259959 

148.80733450633352 

148.80733450633352 



In [26]:
wasserstein_distance_nd(lefthh, righthh)

148.80733450633352

array([[-0.22146015, -1.20012052, -0.6699651 , ...,  0.47964543,
        -2.02497188, -2.52262913],
       [ 0.80358858,  0.42939909,  1.22193065, ...,  0.72460522,
         0.18299241,  0.3622289 ]])

In [21]:
max_tau   = max(abstraction_errors, key=abstraction_errors.get)
max_error = abstraction_errors[max_tau]

print(f"Abstraction: {max_tau}, Error: {max_error}")
max_lenv = max_env_avg_interv_error_key[0]
max_henv = max_env_avg_interv_error_key[1]

Abstraction: [[0.19189168 0.13796931 0.67013901]
 [0.19350908 0.44310058 0.36339034]], Error: 146.88174146110885
